In [61]:
import numpy as np
import tensorflow as tf
import keras
import pandas as pd
import numpy
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import os
import cv2
import random

In [62]:
from tqdm import tqdm
from skimage.feature import graycomatrix, graycoprops
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.layers import Dense, Input,Dropout,GlobalAveragePooling2D,Input,Conv2D, BatchNormalization, Activation, MaxPooling2D,Flatten
from keras.preprocessing import image_dataset_from_directory
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras import layers


In [63]:
dataset_path = 'Dermnet'

In [64]:
#dataset paths
dataset_train_path = os.path.join(dataset_path,'train')
dataset_test_path = os.path.join(dataset_path,'test')



In [65]:
# train and test dataset
dataset_train = tf.keras.preprocessing.image_dataset_from_directory(
 dataset_train_path,
  batch_size=32)

dataset_test = tf.keras.preprocessing.image_dataset_from_directory(
  dataset_test_path,
  batch_size=32
)

Found 344 files belonging to 23 classes.
Found 378 files belonging to 23 classes.


In [66]:
#labels
dataset_name=dataset_train
class_names=dataset_train.class_names

print(class_names)

['Acne and Rosacea Photos', 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions', 'Atopic Dermatitis Photos', 'Bullous Disease Photos', 'Cellulitis Impetigo and other Bacterial Infections', 'Eczema Photos', 'Exanthems and Drug Eruptions', 'Hair Loss Photos Alopecia and other Hair Diseases', 'Herpes HPV and other STDs Photos', 'Light Diseases and Disorders of Pigmentation', 'Lupus and other Connective Tissue diseases', 'Melanoma Skin Cancer Nevi and Moles', 'Nail Fungus and other Nail Disease', 'Poison Ivy Photos and other Contact Dermatitis', 'Psoriasis pictures Lichen Planus and related diseases', 'Scabies Lyme Disease and other Infestations and Bites', 'Seborrheic Keratoses and other Benign Tumors', 'Systemic Disease', 'Tinea Ringworm Candidiasis and other Fungal Infections', 'Urticaria Hives', 'Vascular Tumors', 'Vasculitis Photos', 'Warts Molluscum and other Viral Infections']


In [67]:
#Loading training images

trn_images = []
trn_labels = []
for root, directories, files in os.walk(dataset_train_path):
    for file in files:
        image = cv2.imread(os.path.join(root, file))
        
        #check if image is loaded successfully
        if image is not None:
            image = cv2.resize(image, (224, 224))
            trn_images.append(image)
            trn_labels.append(root.split('/')[-1])
        else:
            print(f"Failed to load image: {os.path.join(root, file)}")
        


#Loading test images
tst_images = []
tst_labels = []

for label in os.listdir(dataset_test_path):
   for file in files:
       image = cv2.imread(os.path.join(root, file))
       
       if image is not None:
        image = cv2.resize(image, (224, 224))
        tst_images.append(image)
        tst_labels.append(root.split('/')[-1])
       else:
          print(f"Failed to load image: {os.path.join(root, file)}")

In [68]:
from keras.preprocessing.image import ImageDataGenerator

#ImageDataGenerator instance
datagen = ImageDataGenerator(rescale=1./255)

#data generators for training and testing
trn_generator = datagen.flow_from_directory(dataset_train_path,
                                            target_size=(224,224),
                                            batch_size=32,
                                            class_mode='binary')
tst_generator = datagen.flow_from_directory(
    dataset_test_path,
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

Found 344 images belonging to 23 classes.
Found 378 images belonging to 23 classes.


In [69]:
#Preprocessing the data
def image_generator(images):
    for image in images:
        yield image.astype('float32') / 255.0
trn_generator = image_generator(trn_images)
tst_generator = image_generator(tst_images)

In [70]:
#Conerting lists to Numpy arrays
trn_images = np.array(trn_images)
tst_images = np.array(tst_images)

batch_size = 100

# Processing training images in batches
for i in range(0, len(trn_images), batch_size):
    batch = trn_images[i:i + batch_size].astype('float32') / 255.0
    for j in range(len(batch)):
        batch[j] = cv2.GaussianBlur(batch[j], (5,5), sigmaX=0)
    
    trn_images[i:i + batch_size] = batch

# Processing testing images in batches
for i in range(0, len(tst_images), batch_size):
    batch = tst_images[i:i + batch_size].astype('float32') / 255.0
    for j in range(len(batch)):
        batch[j] = cv2.GaussianBlur(batch[j], (5,5), sigmaX=0)

    tst_images[i:i + batch_size] = batch


In [71]:
#Resizing
trn_images_resized = [cv2.resize(image, (64, 64)) for image in trn_images]
tst_images_resized = [cv2.resize(image, (64,64))for image in tst_images]

#Reshaping the resized images
trn_images_flat = np.array([image.flatten() for image in trn_images_resized])
tst_images_flat = np.array([image.flatten() for image in tst_images_resized])

#Segmentation with K-Means clustering
trn_images =trn_images.reshape((trn_images.shape[0], -1))
tst_images = tst_images.reshape((tst_images.shape[0], -1))
kmeans = KMeans(n_clusters=2)
kmeans.fit(trn_images)
trn_labels = kmeans.predict(trn_images)
tst_labels = kmeans.predict(tst_images)

c:\Users\Tekkz\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Tekkz\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1152: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


In [72]:
print(image.shape)

(224, 224, 3)


In [73]:
print("Current shape of image array", image.shape)

Current shape of image array (224, 224, 3)


In [74]:
# GLCM Feature extraction
trn_features = []
trn_images = []
tst_images = []

for image in trn_images:
    if len(image.shape) == 1:
        side_length = int(np.sqrt(len(image)))
        # Checking if reshaped dimensions are valid
        if side_length * side_length == len(image):
            image = image.reshape((side_length, side_length))
        else:
            print(f"Skipping image with invalid reshape dimensions: {image.shape}")
            continue
    elif len(image.shape) == 2:
        features = graycomatrix(image, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4])
        trn_features.append(features.flatten())
    else:
        print(f"Skipping image with shape {image.shape}. It must be a 2D array.")

tst_features = []

for image in tst_images:
    print(f"Image shape before graycomatrix: {image.shape}")
    if len(image.shape) == 1:
        side_length = int(np.sqrt(len(image)))
        # Checking if reshaped dimensions are valid
        if side_length * side_length == len(image):
            image = image.reshape((side_length, side_length))
        else:
            print(f"Skipping testing image with invalid reshape dimensions: {image.shape}")
            continue
    elif len(image.shape) == 2:
        features = graycomatrix(image, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4])
        tst_features.append(features.flatten())
    else:
        print(f"Skipping testing image with shape {image.shape}. It must be a 2D array.")



In [75]:
#Converting labels to categorical
trn_labels = to_categorical(trn_labels)
tst_labels = to_categorical(tst_labels)

In [85]:
from keras import models
num_classes = 23

#base pre-trained model
base_model = InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(224,224,3))

#Freezing the base model weights
for layer in base_model.layers:
    layer.trainable = False

In [91]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Assuming num_classes is the number of output classes
])


trn_generator = datagen.flow_from_directory(
    dataset_train_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

#Unfreezing the last few layers
for layer in base_model.layers[-20:]:
    layer.trainable = True



# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Now, you can train the model on your dataset
model.fit(trn_generator, epochs=20)

Found 344 images belonging to 23 classes.
Epoch 1/20
4/4 [==============================] - 53s 9s/step - loss: 3.1976 - accuracy: 0.0901
Epoch 2/20
4/4 [==============================] - 35s 8s/step - loss: 2.4702 - accuracy: 0.3430
Epoch 3/20
4/4 [==============================] - 37s 9s/step - loss: 1.6301 - accuracy: 0.5872
Epoch 4/20
4/4 [==============================] - 37s 8s/step - loss: 0.9317 - accuracy: 0.7674
Epoch 5/20
4/4 [==============================] - 35s 8s/step - loss: 0.4656 - accuracy: 0.8953
Epoch 6/20
4/4 [==============================] - 34s 8s/step - loss: 0.2601 - accuracy: 0.9244
Epoch 7/20
4/4 [==============================] - 34s 8s/step - loss: 0.1623 - accuracy: 0.9448
Epoch 8/20
4/4 [==============================] - 35s 8s/step - loss: 0.1414 - accuracy: 0.9680
Epoch 9/20
4/4 [==============================] - 35s 10s/step - loss: 0.1126 - accuracy: 0.9535
Epoch 10/20
4/4 [==============================] - 34s 8s/step - loss: 0.1111 - accuracy: 0.9

In [77]:
#Defining the Capsnet model
from __future__ import division, print_function, unicode_literals #support both python 2 & 3

import numpy as np
import tensorflow as tf

In [78]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.InteractiveSession(config=config)

c:\Users\Tekkz\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\client\session.py:1793: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [103]:
import tensorflow as tf
from keras import layers, models

# Define the input shape
input_shape = (224, 224, 3)

trn_generator = datagen.flow_from_directory(
    dataset_train_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

tst_generator = datagen.flow_from_directory(
    dataset_test_path,
    target_size=(224,224),
    batch_size=batch_size,
    class_mode='categorical'
)


# Convert trn_labels to one-hot encoding if it's not already
trn_labels_one_hot = tf.keras.utils.to_categorical(trn_labels, num_classes=23)


print("Number of training samples:", len(trn_generator))
print("Number of training labels:", len(trn_labels_one_hot))


# Create the model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Add a Primary Capsule Layer
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Reshape((-1, 8)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Reshape((-1, 32, 8)))

# Add a Digit Capsule Layer
model.add(layers.Conv2D(10, (1, 1), activation='sigmoid'))
model.add(layers.Reshape((-1, 10, 8)))

# Add a fully connected layer
model.add(layers.Flatten())
model.add(layers.Dense(23, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(trn_generator, epochs=10)

tst_images = tst_images.reshape((-1, 224, 224, 3))
tst_labels_one_hot = tf.keras.utils.to_categorical(tst_labels, num_classes=23)

# Evaluate the model
model.evaluate(tst_generator)

Found 344 images belonging to 23 classes.
Found 378 images belonging to 23 classes.
Number of training samples: 4
Number of training labels: 344
Epoch 1/10
4/4 [==============================] - 301s 65s/step - loss: 2187.3459 - accuracy: 0.0349
Epoch 2/10
1/4 [======>.......................] - ETA: 4:17 - loss: 4202.2598 - accuracy: 0.0200

In [82]:
model.save("D:\Documents\ICS Project II\Project II\Skin disease")

NameError: name 'model' is not defined